In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor

In [4]:
import os
import pandas as pd

In [5]:
import torchvision.models as models

In [6]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [7]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [9]:
from sklearn.model_selection import train_test_split

In [23]:
# Define the folder containing your images
fakeThisImage_folder = r'E:\DeepFake\DeepFake\Dataset\Fake-This'
fakeGenImage_folder = r'E:\DeepFake\DeepFake\Dataset\Fake-Gen'
fakeImage_folder = r'E:\DeepFake\DeepFake\Dataset\Fake'
realImage_folder = r'E:\DeepFake\DeepFake\Dataset\Real'

# List all image files in the folder
fakeImage_files = [os.path.join(fakeImage_folder, file) for file in os.listdir(fakeImage_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
fakeThisImage_files = [os.path.join(fakeThisImage_folder, file) for file in os.listdir(fakeThisImage_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
fakeGenImage_files = [os.path.join(fakeGenImage_folder, file) for file in os.listdir(fakeGenImage_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
realImage_files = [os.path.join(realImage_folder, file) for file in os.listdir(realImage_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

fake_df = pd.DataFrame(columns=['Image_Path', 'Fake'])
fake_df['Image_Path'] = fakeImage_files
fake_df['Fake'] = 1

fake_this_df = pd.DataFrame(columns=['Image_Path', 'Fake'])
fake_this_df['Image_Path'] = fakeThisImage_files
fake_this_df['Fake'] = 1

fake_Gen_df = pd.DataFrame(columns=['Image_Path', 'Fake'])
fake_Gen_df['Image_Path'] = fakeGenImage_files
fake_Gen_df['Fake'] = 1

real_df = pd.DataFrame(columns=['Image_Path', 'Fake'])
real_df['Image_Path'] = realImage_files
real_df['Fake'] = 0

result_df = pd.concat([fake_df[70000:71500],fake_this_df,fake_Gen_df, real_df[:3000]],ignore_index=True)

In [25]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7805 entries, 0 to 7804
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Image_Path  7805 non-null   object
 1   Fake        7805 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 122.1+ KB


In [26]:
# Split the data into a training set and a test set (50% each) with class balance
X_train, X_test, y_train, y_test = train_test_split(result_df['Image_Path'], result_df['Fake'], test_size=0.1, stratify= result_df['Fake'], random_state=35)

In [27]:
class ImageModel(nn.Module):
    def __init__(self, num_classes):
        super(ImageModel, self).__init__()
        model = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.DEFAULT)
        
        # Freeze all layers initially
        for param in model.parameters():
            param.requires_grad = False
        
        # Unfreeze the last three layers
        for layer in list(model.children())[-3:]:
            for param in layer.parameters():
                param.requires_grad = True

        self.model = nn.Sequential(*list(model.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.3)
        self.linear1 = nn.Linear(2048, num_classes)

    def forward(self, x):
        batch_size, c, h, w = x.shape
        x = x.view(batch_size, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size, 2048)
        x = self.relu(x)
        x = self.dp(x)
        x = self.linear1(x)
        return fmap, x

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 16

In [29]:
class FaceDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [30]:
transform = transforms.Compose([ transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [31]:
train_dataset = FaceDataset(X_train.values, y_train.values, transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = FaceDataset(X_test.values, y_test.values, transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [44]:
model = ImageModel(num_classes=2)
model.load_state_dict(torch.load(r'E:\DeepFake\DeepFake\model\lstm_model7_0.02736789979094257_0.9897857142857143.pth'))
model = model.to(device)

In [45]:
best_val_acc = 0.0
patience = 5  # Number of epochs to wait for improvement
early_stopping_counter = 0
num_epochs = 50

In [46]:
# Step 3: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [51]:
for epoch in range(num_epochs):
    model.train()
    tt_loss = 0.0
    correct_predictions = 0
    tt_samples = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        _, predictions = model(inputs)
        loss = criterion(predictions, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(predictions, 1)
        correct_predictions += (predicted == labels).sum().item()
        tt_samples += labels.size(0)
        tt_loss += loss.item()

    avg_loss = tt_loss / len(train_loader)
    acc = correct_predictions / tt_samples

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}')

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct_predictions = 0
    val_samples = 0

    with torch.no_grad():
        for val_inputs, val_labels in test_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)

            _, val_predictions = model(val_inputs)
            val_loss += criterion(val_predictions, val_labels).item()

            _, val_predicted = torch.max(val_predictions, 1)
            val_correct_predictions += (val_predicted == val_labels).sum().item()
            val_samples += val_labels.size(0)

    avg_val_loss = val_loss / len(test_loader)
    val_acc = val_correct_predictions / val_samples

    print(f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')

    # Check for improvement in accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        early_stopping_counter = 0
        # Save the model
        model_path = f'E:\DeepFake\DeepFake\model\lstm_model_{epoch}_{avg_loss}_{val_acc}.pth'
        print('Saving model:', model_path)
        torch.save(model.state_dict(), model_path)
    else:
        early_stopping_counter += 1

    # Check for early stopping
    if early_stopping_counter >= patience:
        print(f'Early stopping at epoch {epoch+1} due to lack of improvement in accuracy.')
        break

Epoch [1/50], Loss: 0.0793, Accuracy: 0.9728
Validation Loss: 0.1096, Validation Accuracy: 0.9526
Saving model: E:\DeepFake\DeepFake\model\lstm_model_0_0.07931164814712711_0.9526248399487837.pth
Epoch [2/50], Loss: 0.0555, Accuracy: 0.9781
Validation Loss: 0.1298, Validation Accuracy: 0.9539
Saving model: E:\DeepFake\DeepFake\model\lstm_model_1_0.05548141998034538_0.9539052496798975.pth
Epoch [3/50], Loss: 0.0348, Accuracy: 0.9870
Validation Loss: 0.1516, Validation Accuracy: 0.9488
Epoch [4/50], Loss: 0.0325, Accuracy: 0.9900
Validation Loss: 0.1094, Validation Accuracy: 0.9616
Saving model: E:\DeepFake\DeepFake\model\lstm_model_3_0.03247595861232106_0.9615877080665813.pth
Epoch [5/50], Loss: 0.0278, Accuracy: 0.9899
Validation Loss: 0.1854, Validation Accuracy: 0.9501
Epoch [6/50], Loss: 0.0165, Accuracy: 0.9942
Validation Loss: 0.1510, Validation Accuracy: 0.9718
Saving model: E:\DeepFake\DeepFake\model\lstm_model_5_0.01650273857114437_0.971830985915493.pth
Epoch [7/50], Loss: 0.022

In [49]:
model_path = r'E:\DeepFake\DeepFake\model\resnext_model_{epoch}_{avg_loss}_{val_acc}.pth'
print('Saving model:', model_path)
torch.save(model.state_dict(), model_path)

Saving model: E:\DeepFake\DeepFake\model\resnext_model_{epoch}_{avg_loss}_{val_acc}.pth
